In [1]:
import numpy as np
import pandas as pd
import os

# Train file create

Open the DCADE output TableA.txt and read page file to give each node data corresponding labels (column number).

In [2]:
def train_file_generate(set_total, current_path):
    table_name = os.path.join(current_path, "data", "TableA.txt")
    print("Table Opening:" + table_name + "\n")
    table_a = open(table_name, "r")
    output_name = os.path.join(current_path, "data", "train_raw.txt")
    print("Generating:" + output_name + "\n")

    Set_index = {}

    output = open(output_name, "w")
    output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tPath\tContent\tLabel\n")

    line = table_a.readline()
    slot = line.rstrip("\n").split("\t") # Read Table line in line and split by \t saved in slot.

    while(slot[0]!="ColType"):
        '''
        Loop until we find the "ColType" in the table file to start generate training data.
        '''
        print("find coltype")
        line = table_a.readline()
        slot = line.rstrip("\n").split("\t")
    col = []
    line = table_a.readline() # Read first line
    while(line != ""):
        slot = line.rstrip("\n").split("\t")
        page_name = slot[0]
        node_col = slot[1:] # Save each column's data in node_col
        col = []
        for i in node_col: # Split Same column data and save in list col
            tmp = i.split(" ")
            col.append(tmp)

        current_page = 0
        for pos in range(len(col)): # Loop all col num
            if len(col[pos]) == 1 and not col[pos][0].isnumeric(): # Find "X-X" like sets
                #print("Find : "+col[pos][0])
                current_page = col[pos][0].split("-")[1]
                current_page = int(current_page)
                set_num = col[pos][0].split("-")[0]
                set_num = int(set_num)
                if set_num not in Set_index.keys():
                    #print("Add new set in dict col num:"+str(pos+1))
                    Set_index[set_num] = pos+1 # Record it in dict

        page_file = pd.read_csv(os.path.join(current_path, "Output", page_name), sep='\t') # Read page file from DECADE Output file.

        col_name = []
        for i in page_file.columns: # Save all column name
            col_name.append(i)

        print("open "+ page_name)
        label_dict = {} # Define Dict of labels for each node
        for leafnode in page_file[col_name[0]]: # Loop each node in page
            for pos in range(len(col)): # Loop all col
                if str(leafnode) in col[pos]: # if node is in this column
                    if leafnode not in label_dict.keys(): # and if this node not in dict save it
                        label_dict[leafnode] = pos+1 # Shift one column for every node (zero is for empty).
                    #print(str(leafnode) + "\'s label: " + str(pos))
                    break

        for set_count in range(set_total): # Loop every set and give each node a label, which is the column number of the set.
            with open(os.path.join(current_path, "data", "Set-"+str(set_count+1)+".txt"), "r") as set_file:
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")

                while(slot[0] != "ColType"):
                    line = set_file.readline()
                    slot = line.rstrip("\n").split("\t")

                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
                while(line!=""):
                    file_set_info = slot[0].split("-")
                    file_set_num = set_count
                    file_set_page = int(file_set_info[1])
                    for node in slot[1:]:
                        if not node.isnumeric():
                            continue
                        node = int(node)
                        if file_set_page == current_page:
                            label_dict[node] = Set_index[set_count+1]
                    line = set_file.readline()
                    slot = line.rstrip("\n").split("\t")
        error = False
        '''for i in range(max(label_dict.keys())+1):
            if i not in label_dict:
                error = True
                print(i)'''
        #print(label_dict)
        #count = 0
        data_list = [[], [], [], [], [], [], [], [], []]
        for i in range(max(label_dict.keys())+1): # Loop each recorded node and output as train file
            Leafnode = page_file[col_name[0]][i]
            PTypeSetid = page_file[col_name[6]][i]
            TypeSetid = page_file[col_name[7]][i]
            Contentid = page_file[col_name[8]][i].split("-")[1]
            Pathid = page_file[col_name[9]][i]
            SimSeqid = page_file[col_name[10]][i]
            Path = page_file[col_name[2]][i]
            Content = page_file[col_name[1]][i]
            cols = [Leafnode, PTypeSetid, TypeSetid, Contentid, Pathid, SimSeqid, Path, Content]
            for c in range(len(cols)):
                output.write(str(cols[c]) + "\t")
                data_list[c].append(cols[c])
            if i not in label_dict:
                output.write("0\n")
                data_list[len(cols)].append(0)
            else:
                output.write(str(label_dict[i]) + "\n")
                data_list[len(cols)].append(label_dict[i])
        line = table_a.readline()
    output.close()
    data = pd.DataFrame(np.transpose(np.array(data_list)), 
                 columns=["Leafnode", "PTypeSet", "TypeSet", "Contentid", "Pathid", "Simseqid", "Path", "Content", "Label"]).set_index("Leafnode")
    return data, Set_index

In [3]:
if __name__ == "__main__":
    set_total = 0
    current_path = os.path.join(os.path.expanduser("~"), "jupyter", "web_verification")
    print(current_path)
    data, Set_index = train_file_generate(set_total, current_path)
    if set_total > 0:
        with open(os.path.join(current_path, "data", "Set_idx.txt"), "w") as set_file:
            set_file.write(str(Set_index))
        print(Set_index)

/home/rick/jupyter/web_verification
Table Opening:/home/rick/jupyter/web_verification/data/TableA.txt

Generating:/home/rick/jupyter/web_verification/data/train_raw.txt

find coltype
find coltype
find coltype
find coltype
find coltype
find coltype
open page-0001.txt
open page-0002.txt
open page-0003.txt
open page-0004.txt
open page-0005.txt
open page-0006.txt
open page-0007.txt
open page-0008.txt
open page-0009.txt
open page-0010.txt
open page-0011.txt
open page-0012.txt
open page-0013.txt
open page-0014.txt
open page-0015.txt
open page-0016.txt
open page-0017.txt
open page-0018.txt
open page-0019.txt
open page-0020.txt
open page-0021.txt
open page-0022.txt
open page-0023.txt
open page-0024.txt
open page-0025.txt
open page-0026.txt
open page-0027.txt
open page-0028.txt
open page-0029.txt
open page-0030.txt


In [4]:
data

,PTypeSet,TypeSet,Contentid,Pathid,Simseqid,Path,Content,Label
Leafnode,,,,,,,,
0,1,31,1801,0,0,"[HTML, HEAD, TITLE, #text]",Offshore Energy 2011 on LinkedIn Events,1
1,3,4,2,1,0,"[HTML, HEAD, LINK]",http://events-cdn.linkedin.com/stylesheets/sty...,2
2,3,4,3,1,0,"[HTML, HEAD, LINK]",http://events-cdn.linkedin.com/stylesheets/api...,3
3,3,4,4,2,1,"[HTML, BODY, DIV, DIV, H1, A, IMG]",http://static.linkedin.com/img/pic/pic_logo_11...,4
4,3,4,5,2,2,"[HTML, BODY, DIV, DIV, H1, A]",http://www.linkedin.com/?trk=hb_logo,5
...,...,...,...,...,...,...,...,...
141,3,4,80,3,4,"[HTML, BODY, DIV, DIV, UL, LI, A]",http://www.linkedin.com/static?key=copyright_p...,78
142,5,27,81,8,15,"[HTML, BODY, DIV, DIV, P, #text]",Use of this site is subject to express,79
143,5,30,82,8,16,"[HTML, BODY, DIV, DIV, P, A, #text]",terms of use,80
